In [60]:
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets
import os
import torch.optim as optim
import torch.nn as nn
import torchvision.models as models
import random
from tqdm import tqdm

# Define dataset path
# data_dir = 'Images_Dataset/train'
data_dir = 'cards_dataset/train'


In [61]:

# Define the data transformations
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Directories for the dataset
train_dir = 'cards_dataset/train'
test_dir = 'cards_dataset/test'
rehearsal_dir = 'cards_dataset/rehearsal'  # Unified directory for rehearsal buffer

# Load datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=data_transforms)
test_dataset = datasets.ImageFolder(root=test_dir, transform=data_transforms)


In [62]:

# Define data transforms
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Load dataset
train_dataset = datasets.ImageFolder(root=data_dir, transform=data_transforms)

# Set number of workers for data loading
num_workers = 4

# Load data with multi-threaded data loading
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=num_workers)

# Load pre-trained EfficientNet model
model = models.efficientnet_b0(pretrained=True)

# Modify the classifier layer to match the number of classes in our dataset
num_classes = len(train_dataset.classes)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)


In [63]:

class RehearsalBuffer:
    def __init__(self, buffer_size):
        self.buffer_size = buffer_size
        self.buffer = []
    
    def add_data(self, data):
        self.buffer.extend(data)
        if len(self.buffer) > self.buffer_size:
            self.buffer = random.sample(self.buffer, self.buffer_size)
    
    def sample_data(self, sample_size):
        return random.sample(self.buffer, min(sample_size, len(self.buffer)))

# Initialize rehearsal buffer
buffer_size = 500  # Set buffer size
rehearsal_buffer = RehearsalBuffer(buffer_size)

# Training parameters
num_epochs = 10
#learning_rate = 0.001
learning_rate = 0.000707

sample_size = 64  # Number of samples to draw from the rehearsal buffer

# Loss and optimizer
criterion = nn.CrossEntropyLoss()

# Use AdamW optimizer
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)


In [64]:

# # Training loop
# for epoch in range(num_epochs):
#     model.train()
#     running_loss = 0.0
    
#     # Wrap train_loader with tqdm for the progress bar
#     train_loader_with_progress = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}', leave=False)
    
#     for images, labels in train_loader_with_progress:
#         # Add current batch to the rehearsal buffer
#         rehearsal_buffer.add_data(list(zip(images, labels)))
        
#         # Sample from the rehearsal buffer
#         if len(rehearsal_buffer.buffer) > 0:
#             buffer_samples = rehearsal_buffer.sample_data(sample_size)
#             buffer_images, buffer_labels = zip(*buffer_samples)
#             buffer_images = torch.stack(buffer_images)
#             buffer_labels = torch.tensor(buffer_labels)
            
#             # Combine current batch and buffer samples
#             combined_images = torch.cat((images, buffer_images))
#             combined_labels = torch.cat((labels, buffer_labels))
#         else:
#             combined_images = images
#             combined_labels = labels
        
#         # Zero the parameter gradients
#         optimizer.zero_grad()
        
#         # Forward pass
#         outputs = model(combined_images)
#         loss = criterion(outputs, combined_labels)
        
#         # Backward pass and optimize
#         loss.backward()
#         optimizer.step()
        
#         running_loss += loss.item()
        
#         # Update the progress bar with the current loss
#         train_loader_with_progress.set_postfix({'Loss': loss.item()})
    
#     print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader)}')

# print('Training completed.')


In [65]:
# # Define the path where you want to save the model
# model_save_path = 'efficientnet_cl_model.pth'

# # Save the model state dictionary
# torch.save(model.state_dict(), model_save_path)

# print(f'Model saved to {model_save_path}')


In [66]:
import torch
import torch.nn as nn
import torchvision.models as models

# Define the path from where you want to load the CL model
model_load_path = 'efficientnet_cl_model_ORIG.pth'

# Load the EfficientNet model architecture
model = models.efficientnet_b0(pretrained=False)  # Set pretrained=False to load the architecture without pretrained weights

# Modify the classifier layer to match the number of classes in your dataset
# (Make sure to adjust this based on your specific dataset)
num_classes = len(train_dataset.classes)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)

# Load the state dictionary from the file specified by model_load_path into the model
model.load_state_dict(torch.load(model_load_path))

# Set the model to evaluation mode
model.eval()

print(f'CL Model loaded from {model_load_path}')






CL Model loaded from efficientnet_cl_model_ORIG.pth


In [67]:
import os
import shutil

# Define the source and destination directories
rehearsal_dir = 'cards_dataset/rehearsal'
ignore_dir = 'cards_dataset/ignore'

# Create the ignore directory if it doesn't exist
os.makedirs(ignore_dir, exist_ok=True)

# Get all subdirectories in the rehearsal directory
subdirs = [d for d in os.listdir(rehearsal_dir) if os.path.isdir(os.path.join(rehearsal_dir, d))]

# Move subdirectories from index 37 onwards to the ignore directory
for idx, subdir in enumerate(subdirs):
    if idx >= 37:
        src_path = os.path.join(rehearsal_dir, subdir)
        dst_path = os.path.join(ignore_dir, subdir)
        
        # Remove existing destination directory if it exists
        if os.path.exists(dst_path):
            shutil.rmtree(dst_path)
        
        # Move the directory
        shutil.move(src_path, dst_path)
        print(f"Moved {src_path} to {dst_path}")

print("Folders moved successfully.")


Moved cards_dataset/rehearsal\ace of diamonds to cards_dataset/ignore\ace of diamonds
Moved cards_dataset/rehearsal\ace of hearts to cards_dataset/ignore\ace of hearts
Moved cards_dataset/rehearsal\ace of spades to cards_dataset/ignore\ace of spades
Moved cards_dataset/rehearsal\eight of clubs to cards_dataset/ignore\eight of clubs
Moved cards_dataset/rehearsal\eight of diamonds to cards_dataset/ignore\eight of diamonds
Moved cards_dataset/rehearsal\eight of hearts to cards_dataset/ignore\eight of hearts
Moved cards_dataset/rehearsal\eight of spades to cards_dataset/ignore\eight of spades
Folders moved successfully.


In [68]:
output_neurons = model.classifier[1].out_features
print(f"Number of output neurons: {output_neurons}")


Number of output neurons: 37


In [69]:
import torch
from tqdm import tqdm
from torch.utils.data import Subset, DataLoader
from torch.optim.lr_scheduler import CyclicLR
from sklearn.model_selection import train_test_split
from torchvision import datasets, transforms
import random
import os
import shutil

# Define the data transformations
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Directories for the dataset
test_dir = 'cards_dataset/test'
rehearsal_dir = 'cards_dataset/rehearsal'  # Unified directory for rehearsal buffer
test_dataset = datasets.ImageFolder(root=test_dir, transform=data_transforms)

# Function to load rehearsal data from a directory
def load_rehearsal_data(root_dir, transform):
    if not os.listdir(root_dir):  # Check if the directory is empty
        return None
    return datasets.ImageFolder(root=root_dir, transform=transform)

# Load the rehearsal dataset
rehearsal_dataset = load_rehearsal_data(rehearsal_dir, data_transforms)

# Split the testing dataset into fine-tuning and evaluation subsets
fine_tune_size = 0.1  # Use 10% for fine-tuning
fine_tune_indices, eval_indices = train_test_split(list(range(len(test_dataset))), test_size=fine_tune_size)
fine_tune_subset = Subset(test_dataset, fine_tune_indices)
eval_subset = Subset(test_dataset, eval_indices)

# Create data loaders for fine-tuning and evaluation subsets
fine_tune_loader = DataLoader(fine_tune_subset, batch_size=64, shuffle=True, num_workers=4)
eval_loader = DataLoader(eval_subset, batch_size=64, shuffle=False, num_workers=4)

# Set up fine-tuning parameters
fine_tune_epochs = 8
fine_tune_learning_rate = 0.00045
fine_tune_optimizer = torch.optim.AdamW(model.parameters(), lr=fine_tune_learning_rate)

# Set up the CyclicLR scheduler
base_lr = 0.0001
max_lr = 0.001
fine_tune_scheduler = CyclicLR(fine_tune_optimizer, base_lr=base_lr, max_lr=max_lr, step_size_up=5, mode='triangular')

fine_tune_criterion = torch.nn.CrossEntropyLoss()

class RehearsalBuffer:
    def __init__(self, rehearsal_dataset):
        self.buffer = []
        self.rehearsal_dataset = rehearsal_dataset

    def add_random_data(self, sample_per_class, sample_per_new_class):
        class_indices = {}
        
        # Collect indices of each class in rehearsal_dataset
        for i, (_, label) in enumerate(self.rehearsal_dataset):
            if label not in class_indices:
                class_indices[label] = []
            class_indices[label].append(i)
        
        # Shuffle and select random samples from each class in rehearsal_dataset
        for class_index, indices in class_indices.items():
            random.shuffle(indices)
            if class_index < 37:
                selected_indices = indices[:sample_per_class]
            else:
                selected_indices = indices[:sample_per_new_class]
            self.buffer.extend([self.rehearsal_dataset[i] for i in selected_indices])

    def sample_data(self, sample_size):
        sample_size = min(sample_size, len(self.buffer))
        return random.sample(self.buffer, sample_size)

# Define the number of samples per class
sample_per_class = 2
sample_per_new_class = 10

for i in range(0, 7):
    # Load the updated rehearsal dataset within the loop
    rehearsal_dataset = load_rehearsal_data(rehearsal_dir, data_transforms)
    
    # Initialize the rehearsal buffer with the updated datasets
    rehearsal_buffer = RehearsalBuffer(rehearsal_dataset)

    # Get the first new class
    first_new_class = test_dataset.classes[i]

    # Function to update the model for new classes
    def add_new_classes_to_model(model, new_classes_count):
        old_num_classes = model.classifier[1].out_features
        new_num_classes = old_num_classes + new_classes_count
        
        # Get the weight and bias of the old classifier layer
        old_weight = model.classifier[1].weight.data
        old_bias = model.classifier[1].bias.data
        
        # Create a new classifier layer with updated number of classes
        new_classifier = torch.nn.Linear(model.classifier[1].in_features, new_num_classes)
        
        # Initialize the new classifier layer with old weights and biases
        new_classifier.weight.data[:old_num_classes] = old_weight
        new_classifier.bias.data[:old_num_classes] = old_bias
        
        # Replace the old classifier with the new one
        model.classifier[1] = new_classifier

    # Add new class to the model before starting the training loop
    add_new_classes_to_model(model, 1)  # Increment by one class

    # Iterate over epochs
    for epoch in range(fine_tune_epochs):
        model.train()
        running_loss = 0.0
        
        # Load data for the new class
        class_index = test_dataset.class_to_idx[first_new_class]
        class_indices = [i for i, (_, label) in enumerate(test_dataset) if label == class_index]
        new_class_subset = Subset(test_dataset, class_indices)
        new_class_loader = DataLoader(new_class_subset, batch_size=64, shuffle=True, num_workers=4)
        
        # Wrap the data loader with tqdm for the progress bar
        new_class_loader_with_progress = tqdm(new_class_loader, desc=f'Epoch {epoch + 1}/{fine_tune_epochs} - Class {first_new_class}', leave=False)
        
        for images, labels in new_class_loader_with_progress:
            # Adjust labels for the new combined classifier
            labels += 37  # Shift labels to the correct position
            
            # Add samples from each class to the rehearsal buffer
            rehearsal_buffer.add_random_data(sample_per_class, sample_per_new_class)
            
            # Sample from the rehearsal buffer
            if len(rehearsal_buffer.buffer) > 0:
                buffer_samples = rehearsal_buffer.sample_data(sample_per_class + sample_per_new_class)
                buffer_images, buffer_labels = zip(*buffer_samples)
                buffer_images = torch.stack(buffer_images)
                buffer_labels = torch.tensor(buffer_labels)
                
                # Combine current batch and buffer samples
                combined_images = torch.cat((images, buffer_images))
                combined_labels = torch.cat((labels, buffer_labels))
            else:
                combined_images = images
                combined_labels = labels
            
            # Zero the parameter gradients
            fine_tune_optimizer.zero_grad()
            
            # Forward pass
            outputs = model(combined_images)
            loss = fine_tune_criterion(outputs, combined_labels)
            
            # Backward pass and optimize
            loss.backward()
            fine_tune_optimizer.step()

            # Step the CyclicLR scheduler
            fine_tune_scheduler.step()
            
            running_loss += loss.item()
        
        # Print loss at the end of each epoch
        print(f"Epoch {epoch + 1}/{fine_tune_epochs}, Loss: {running_loss}")

    # Copy the new class images to the rehearsal folder at the end of training

    def copy_new_class_to_rehearsal(test_dir, rehearsal_dir, new_class_name, class_index):
        new_class_dir = os.path.join(test_dir, new_class_name)
        rehearsal_class_dir = os.path.join(rehearsal_dir, new_class_name)
        
        if not os.path.exists(rehearsal_class_dir):
            os.makedirs(rehearsal_class_dir)
        
        for filename in os.listdir(new_class_dir):
            source_path = os.path.join(new_class_dir, filename)
            destination_path = os.path.join(rehearsal_class_dir, filename)
            shutil.copy(source_path, destination_path)

    # Execute the copy function
    copy_new_class_to_rehearsal(test_dir, rehearsal_dir, first_new_class, i + 37)


Epoch 1/5, Loss: 6.314264327287674


Epoch 2/5, Loss: 3.141158774495125


Epoch 3/5, Loss: 1.4734115041792393


Epoch 4/5, Loss: 1.1331035271286964


Epoch 5/5, Loss: 0.8770162016153336


...

Epoch 1/5, Loss: 5.409898400306702


Epoch 2/5, Loss: 4.727463722229004


Epoch 3/5, Loss: 3.405801773071289


Epoch 4/5, Loss: 2.390485107898712


Epoch 5/5, Loss: 2.0645902156829834


In [31]:
# import torch
# from tqdm import tqdm
# from torch.utils.data import Subset, DataLoader
# from torch.optim.lr_scheduler import ReduceLROnPlateau
# from sklearn.model_selection import train_test_split
# from torchvision import datasets, transforms
# import random
# import os
# import shutil

# # Define the data transformations
# data_transforms = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
# ])

# # Directories for the dataset
# test_dir = 'cards_dataset/test'
# rehearsal_dir = 'cards_dataset/rehearsal'  # Unified directory for rehearsal buffer
# test_dataset = datasets.ImageFolder(root=test_dir, transform=data_transforms)

# # Function to load rehearsal data from a directory
# def load_rehearsal_data(root_dir, transform):
#     if not os.listdir(root_dir):  # Check if the directory is empty
#         return None
#     return datasets.ImageFolder(root=root_dir, transform=transform)

# # Load the rehearsal dataset
# rehearsal_dataset = load_rehearsal_data(rehearsal_dir, data_transforms)

# # Split the testing dataset into fine-tuning and evaluation subsets
# fine_tune_size = 0.1  # Use 20% for fine-tuning
# fine_tune_indices, eval_indices = train_test_split(list(range(len(test_dataset))), test_size=fine_tune_size)
# fine_tune_subset = Subset(test_dataset, fine_tune_indices)
# eval_subset = Subset(test_dataset, eval_indices)

# # Create data loaders for fine-tuning and evaluation subsets
# fine_tune_loader = DataLoader(fine_tune_subset, batch_size=64, shuffle=True, num_workers=4)
# eval_loader = DataLoader(eval_subset, batch_size=64, shuffle=False, num_workers=4)

# # Set up fine-tuning parameters
# fine_tune_epochs = 5
# fine_tune_learning_rate = 0.00045
# fine_tune_optimizer = torch.optim.AdamW(model.parameters(), lr=fine_tune_learning_rate)
# fine_tune_criterion = torch.nn.CrossEntropyLoss()
# fine_tune_scheduler = ReduceLROnPlateau(fine_tune_optimizer, mode='min', factor=0.1, patience=3, verbose=True)

# class RehearsalBuffer:
#     def __init__(self, rehearsal_dataset):
#         self.buffer = []
#         self.rehearsal_dataset = rehearsal_dataset

#     def add_random_data(self, sample_per_class, sample_per_new_class):
#         class_indices = {}
        
#         # Collect indices of each class in rehearsal_dataset
#         for i, (_, label) in enumerate(self.rehearsal_dataset):
#             if label not in class_indices:
#                 class_indices[label] = []
#             class_indices[label].append(i)
        
#         # Shuffle and select random samples from each class in rehearsal_dataset
#         for class_index, indices in class_indices.items():
#             random.shuffle(indices)
#             if class_index < 37:
#                 selected_indices = indices[:sample_per_class]
#             else:
#                 selected_indices = indices[:sample_per_new_class]
#             self.buffer.extend([self.rehearsal_dataset[i] for i in selected_indices])

#     def sample_data(self, sample_size):
#         sample_size = min(sample_size, len(self.buffer))
#         return random.sample(self.buffer, sample_size)

# # Define the number of samples per class
# sample_per_class = 2
# sample_per_new_class = 44

# for i in range(0, 3):
#     # Load the updated rehearsal dataset within the loop
#     rehearsal_dataset = load_rehearsal_data(rehearsal_dir, data_transforms)
    
#     # Initialize the rehearsal buffer with the updated datasets
#     rehearsal_buffer = RehearsalBuffer(rehearsal_dataset)

#     # Get the first new class
#     first_new_class = test_dataset.classes[i]

#     # Function to update the model for new classes
#     def add_new_classes_to_model(model, new_classes_count):
#         old_num_classes = model.classifier[1].out_features
#         new_num_classes = old_num_classes + new_classes_count
        
#         # Get the weight and bias of the old classifier layer
#         old_weight = model.classifier[1].weight.data
#         old_bias = model.classifier[1].bias.data
        
#         # Create a new classifier layer with updated number of classes
#         new_classifier = torch.nn.Linear(model.classifier[1].in_features, new_num_classes)
        
#         # Initialize the new classifier layer with old weights and biases
#         new_classifier.weight.data[:old_num_classes] = old_weight
#         new_classifier.bias.data[:old_num_classes] = old_bias
        
#         # Replace the old classifier with the new one
#         model.classifier[1] = new_classifier

#     # Add new class to the model before starting the training loop
#     add_new_classes_to_model(model, 1)  # Increment by one class

#     # Iterate over epochs
#     for epoch in range(fine_tune_epochs):
#         model.train()
#         running_loss = 0.0
        
#         # Load data for the new class
#         class_index = test_dataset.class_to_idx[first_new_class]
#         class_indices = [i for i, (_, label) in enumerate(test_dataset) if label == class_index]
#         new_class_subset = Subset(test_dataset, class_indices)
#         new_class_loader = DataLoader(new_class_subset, batch_size=64, shuffle=True, num_workers=4)
        
#         # Wrap the data loader with tqdm for the progress bar
#         new_class_loader_with_progress = tqdm(new_class_loader, desc=f'Epoch {epoch + 1}/{fine_tune_epochs} - Class {first_new_class}', leave=False)
        
#         for images, labels in new_class_loader_with_progress:
#             # Adjust labels for the new combined classifier
#             labels += 37  # Shift labels to the correct position
            
#             # Add samples from each class to the rehearsal buffer
#             rehearsal_buffer.add_random_data(sample_per_class, sample_per_new_class)
            
#             # Sample from the rehearsal buffer
#             if len(rehearsal_buffer.buffer) > 0:
#                 buffer_samples = rehearsal_buffer.sample_data(sample_per_class + sample_per_new_class)
#                 buffer_images, buffer_labels = zip(*buffer_samples)
#                 buffer_images = torch.stack(buffer_images)
#                 buffer_labels = torch.tensor(buffer_labels)
                
#                 # Combine current batch and buffer samples
#                 combined_images = torch.cat((images, buffer_images))
#                 combined_labels = torch.cat((labels, buffer_labels))
#             else:
#                 combined_images = images
#                 combined_labels = labels
            
#             # Zero the parameter gradients
#             fine_tune_optimizer.zero_grad()
            
#             # Forward pass
#             outputs = model(combined_images)
#             loss = fine_tune_criterion(outputs, combined_labels)
            
#             # Backward pass and optimize
#             loss.backward()
#             fine_tune_optimizer.step()
            
#             running_loss += loss.item()
        
#         # Reduce learning rate if validation loss plateaus
#         fine_tune_scheduler.step(running_loss)
        
#         # Print loss at the end of each epoch
#         print(f"Epoch {epoch + 1}/{fine_tune_epochs}, Loss: {running_loss}")

#     # Copy the new class images to the rehearsal folder at the end of training

#     def copy_new_class_to_rehearsal(test_dir, rehearsal_dir, new_class_name, class_index):
#         new_class_dir = os.path.join(test_dir, new_class_name)
#         rehearsal_class_dir = os.path.join(rehearsal_dir, new_class_name)
        
#         if not os.path.exists(rehearsal_class_dir):
#             os.makedirs(rehearsal_class_dir)
        
#         for filename in os.listdir(new_class_dir):
#             source_path = os.path.join(new_class_dir, filename)
#             destination_path = os.path.join(rehearsal_class_dir, filename)
#             shutil.copy(source_path, destination_path)
        
#         # Rename the rehearsal class folder to include the class index
#         # new_rehearsal_class_dir = os.path.join(rehearsal_dir, f"{class_index:02d}_{new_class_name}")
#         # os.rename(rehearsal_class_dir, new_rehearsal_class_dir)


#     # Execute the copy function
#     copy_new_class_to_rehearsal(test_dir, rehearsal_dir, first_new_class, i + 37)


Epoch 1/5, Loss: 6.434596538543701


Epoch 2/5, Loss: 2.628715991973877


Epoch 3/5, Loss: 1.0855591744184494


Epoch 4/5, Loss: 0.7600512206554413


Epoch 5/5, Loss: 0.46160848438739777


In [70]:
train_dataset = datasets.ImageFolder(root='cards_dataset/train', transform=data_transforms)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False, num_workers=4)

model.eval()
total_correct = 0
total_samples = 0

with torch.no_grad():
    for images, labels in train_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()

train_accuracy = total_correct / total_samples
print(f"Accuracy on train dataset: {train_accuracy:.2%}")


Accuracy on train dataset: 96.56%


In [71]:
import torch
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
import numpy as np

# Define the data transformations
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Directories for the dataset
test_dir = 'cards_dataset/test'
test_dataset = datasets.ImageFolder(root=test_dir, transform=data_transforms)

# Define DataLoader for the entire test dataset
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

# Load the trained model
model.to(torch.device("cpu"))
model.eval()

# Define the loss function
criterion = torch.nn.CrossEntropyLoss()

overall_correct = 0
overall_counted = 0

# Function to evaluate model on a specific class subset
def evaluate_class(class_name, class_idx, dataset):
    class_indices = [i for i, (_, label) in enumerate(dataset) if label == class_idx]
    class_subset = Subset(dataset, class_indices)
    class_loader = DataLoader(class_subset, batch_size=64, shuffle=False, num_workers=4)
    
    correct = 0
    total = 0
    total_loss = 0.0
    all_predictions = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in class_loader:
            outputs = model(images)
            loss = criterion(outputs, labels + 37)  # Offset labels by 37
            total_loss += loss.item()
            
            _, predicted = torch.max(outputs, 1)
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend((labels + 37).cpu().numpy())  # Offset labels by 37
            total += labels.size(0)
            correct += (predicted == (labels + 37)).sum().item()
    
    accuracy = 100 * correct / total
    average_loss = total_loss / len(class_loader)
    
    return accuracy, average_loss, all_predictions, all_labels

# Iterate through each class in the test dataset
for i in range(0, 7):
    class_name = test_dataset.classes[i]
    class_idx = test_dataset.class_to_idx[class_name]
    
    accuracy_class, avg_loss_class, predictions_class, actuals_class = evaluate_class(class_name, class_idx, test_dataset)
    
    print(f"Accuracy on '{class_name}' dataset: {accuracy_class:.2f}%")
    print(f"Average loss on '{class_name}' dataset: {avg_loss_class:.4f}")
    
    # Convert predictions and actual labels to NumPy arrays
    predictions_class = np.array(predictions_class)
    actuals_class = np.array(actuals_class)
    
    # Update overall metrics
    overall_correct += (predictions_class == actuals_class).sum()
    overall_counted += len(actuals_class)
    
    # # Print predictions along with actual labels for the current class subset
    # print(f"Predictions on '{class_name}' dataset:")
    # for idx, (pred, actual) in enumerate(zip(predictions_class, actuals_class)):
    #     print(f"Sample {idx}: Predicted class {pred}, Actual class {actual}")

# Print overall accuracy across all classes
overall_accuracy = 100 * overall_correct / overall_counted
print(f"Overall accuracy across all classes: {overall_accuracy:.2f}%")


Accuracy on 'ace of diamonds' dataset: 99.22%
Average loss on 'ace of diamonds' dataset: 0.1098
Accuracy on 'ace of hearts' dataset: 99.42%
Average loss on 'ace of hearts' dataset: 0.1030
Accuracy on 'ace of spades' dataset: 96.69%
Average loss on 'ace of spades' dataset: 0.3991
Accuracy on 'eight of clubs' dataset: 97.10%
Average loss on 'eight of clubs' dataset: 0.4256
Accuracy on 'eight of diamonds' dataset: 76.10%
Average loss on 'eight of diamonds' dataset: 1.0330
Accuracy on 'eight of hearts' dataset: 98.03%
Average loss on 'eight of hearts' dataset: 0.3053
Accuracy on 'eight of spades' dataset: 100.00%
Average loss on 'eight of spades' dataset: 0.1349
Overall accuracy across all classes: 95.02%


In [ ]:
# import torch
# from torch.utils.data import Subset, DataLoader
# from torchvision import datasets, transforms
# from PIL import Image
# import os

# # Define the data transformations
# data_transforms = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
# ])

# # Directories for the dataset
# train_dir = 'cards_dataset/test'
# train_dataset = datasets.ImageFolder(root=train_dir, transform=data_transforms)

# # Get the class index for ""
# class_name = "ace of diamonds"
# class_idx = train_dataset.class_to_idx[class_name]

# # Filter indices for the "two of spades" class
# class_indices = [i for i, (_, label) in enumerate(train_dataset) if label == class_idx]

# # Create a subset and DataLoader for the "two of spades" class
# class_subset = Subset(train_dataset, class_indices)
# class_loader = DataLoader(class_subset, batch_size=64, shuffle=False, num_workers=4)

# # Define the evaluation function
# def evaluate_model(model, data_loader, criterion):
#     model.eval()
#     correct = 0
#     total = 0
#     total_loss = 0.0
#     all_predictions = []
#     all_labels = []
    
#     with torch.no_grad():
#         for images, labels in data_loader:
#             outputs = model(images)
#             loss = criterion(outputs, labels)
#             total_loss += loss.item()
            
#             _, predicted = torch.max(outputs, 1)
#             all_predictions.extend(predicted.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())
#             total += labels.size(0)
#             correct += (predicted == 37).sum().item()
    
#     accuracy = 100 * correct / total
#     average_loss = total_loss / len(data_loader)
#     return accuracy, average_loss, all_predictions, all_labels

# # Evaluate the model on the "ace of diamonds" subset
# accuracy_class, avg_loss_class, predictions_class, actuals_class = evaluate_model(model, class_loader, fine_tune_criterion)

# print(f"Accuracy on 'ace of diamonds' dataset: {accuracy_class:.2f}%")
# print(f"Average loss on 'ace of diamonds' dataset: {avg_loss_class:.4f}")

# # Print predictions along with actual labels for the "two of spades" dataset
# print("Predictions on 'ace of diamonds' dataset:")
# for idx, (pred, actual) in enumerate(zip(predictions_class, actuals_class)):
#     print(f"Sample {idx}: Predicted class {pred}, Actual class {actual + 37}")


KeyboardInterrupt: 

In [63]:
# import torch
# from torch.utils.data import Subset, DataLoader
# from torchvision import datasets, transforms
# from PIL import Image
# import os

# # Define the data transformations
# data_transforms = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
# ])

# # Directories for the dataset
# train_dir = 'cards_dataset/test'
# train_dataset = datasets.ImageFolder(root=train_dir, transform=data_transforms)

# # Get the class index for ""
# class_name = "ace of hearts"
# class_idx = train_dataset.class_to_idx[class_name]

# # Filter indices for the "two of spades" class
# class_indices = [i for i, (_, label) in enumerate(train_dataset) if label == class_idx]

# # Create a subset and DataLoader for the "two of spades" class
# class_subset = Subset(train_dataset, class_indices)
# class_loader = DataLoader(class_subset, batch_size=64, shuffle=False, num_workers=4)

# # Define the evaluation function
# def evaluate_model(model, data_loader, criterion):
#     model.eval()
#     correct = 0
#     total = 0
#     total_loss = 0.0
#     all_predictions = []
#     all_labels = []
    
#     with torch.no_grad():
#         for images, labels in data_loader:
#             outputs = model(images)
#             loss = criterion(outputs, labels)
#             total_loss += loss.item()
            
#             _, predicted = torch.max(outputs, 1)
#             all_predictions.extend(predicted.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())
#             total += labels.size(0)
#             correct += (predicted == 38).sum().item()
    
#     accuracy = 100 * correct / total
#     average_loss = total_loss / len(data_loader)
#     return accuracy, average_loss, all_predictions, all_labels

# # Evaluate the model on the "ace of diamonds" subset
# accuracy_class, avg_loss_class, predictions_class, actuals_class = evaluate_model(model, class_loader, fine_tune_criterion)

# print(f"Accuracy on 'ace of hearts' dataset: {accuracy_class:.2f}%")
# print(f"Average loss on 'ace of hearts' dataset: {avg_loss_class:.4f}")

# # Print predictions along with actual labels for the "two of spades" dataset
# print("Predictions on 'ace of hearts' dataset:")
# for idx, (pred, actual) in enumerate(zip(predictions_class, actuals_class)):
#     print(f"Sample {idx}: Predicted class {pred}, Actual class {actual + 37}")


Accuracy on 'ace of hearts' dataset: 90.06%
Average loss on 'ace of hearts' dataset: 11.1137
Predictions on 'ace of hearts' dataset:
Sample 0: Predicted class 38, Actual class 38
Sample 1: Predicted class 38, Actual class 38
Sample 2: Predicted class 38, Actual class 38
Sample 3: Predicted class 38, Actual class 38
Sample 4: Predicted class 38, Actual class 38
Sample 5: Predicted class 38, Actual class 38
Sample 6: Predicted class 38, Actual class 38
Sample 7: Predicted class 38, Actual class 38
Sample 8: Predicted class 38, Actual class 38
Sample 9: Predicted class 38, Actual class 38
Sample 10: Predicted class 37, Actual class 38
Sample 11: Predicted class 38, Actual class 38
Sample 12: Predicted class 38, Actual class 38
Sample 13: Predicted class 38, Actual class 38
Sample 14: Predicted class 38, Actual class 38
Sample 15: Predicted class 38, Actual class 38
Sample 16: Predicted class 38, Actual class 38
Sample 17: Predicted class 38, Actual class 38
Sample 18: Predicted class 38, 